In [1]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

# URL of the website with the table
url = 'https://pokemondb.net/pokedex/all'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table element based on its tag or attributes
# For example, if the table has a specific ID, you can find it like this:
table = soup.find('table', id='pokedex')

# If the table doesn't have an ID, you can find it based on its class or other attributes:
# table = soup.find('table', class_='table_class')

# Create a new Excel workbook
wb = Workbook()
ws = wb.active

# Extract column names from table headers
if table:
    # Find all table headers
    headers = table.find_all('th')
    column_names = [header.get_text(strip=True) for header in headers]
    
    # Modify column names to include type_1 and type_2
    column_names.insert(3, 'Type_1')
    column_names.insert(4, 'Type_2')
    
    # Write column names as the first row in the Excel sheet
    ws.append(column_names)
    
    # Find all table rows
    rows = table.find_all('tr')
    
    # Iterate over table rows
    for row in rows:
        # Extract data cells from each row
        cells = row.find_all('td')
        
        # Ensure that there are enough cells
        if len(cells) >= 3:
            row_data = [cell.get_text(strip=True) for cell in cells]
            
            # Extract type data from the anchor tags
            type_cells = cells[2].find_all('a', class_='type-icon')
            types = [type_cell.get_text(strip=True) for type_cell in type_cells]
            
            # If only one type is present, insert null in the second type column
            if len(types) == 1:
                types.append('null')
            
            # Insert type data into the row data
            row_data[2:3] = types
            
            # Write the modified row data to the Excel sheet
            ws.append(row_data)
else:
    print("Table not found on the page.")

# Save the Excel workbook
wb.save("pokemon_data.xlsx")
